In [ ]:
%matplotlib notebook

import matplotlib
import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from ipywidgets import interact, interactive, fixed, FloatSlider, widgets

Overview
======

This notebook introduces a number of concepts from molecular mechanics: Lennard-Jones potentials, Verlet integration algorithms and periodic boundary conditions.  The internal consistency of SI units make life easier when implementing equations, but we'll use kJ/mol, nm, a.m.u. and ps units for energy, distance, mass and time, respectively.  In these units, forces on particle $i$ at the resulting from $F_i(\mathbf{r})=m_ia_i(\mathbf{r})$ and $F_i(\mathbf{r})=-\nabla_i U(\mathbf{r})$, where $U(\mathbf{r})$ is the potential energy at configuration $\mathbf{r}$, are consistent from the perspective of units.

The Lennard-Jones Potential
-------------------------
Given the well depth $\epsilon$ (units of kJ/mol) and a parameter $\sigma$ that is related to the radius of an atom, the Lennard-Jones (LJ) potential for a pair of atoms separated by $r$ is given by

\begin{equation}
U_{LJ}(r) = 4\epsilon\left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^6\right].
\end{equation}

The following plot shows the LJ energy, as a function of distance.

In [ ]:
def showpotential(ax, sigma=0.25, epsilon=0.05):
    """ Plot the Lennard-Jones potential for a pair,
        given the sigma and epsilon values. """
    npts = 200
    sigmapair = 2*sigma # combine the sigmas on each atom
    # Go up to 15A, using npts points
    maxr = 1.5
    rvals = np.linspace(0.015, maxr, npts)
    sig_r = sigmapair / rvals
    sig_r2 = np.square(sig_r)
    sig_r6 = sig_r2*sig_r2*sig_r2
    sig_r12 = sig_r6*sig_r6
    energies = 4*epsilon*(sig_r12 - sig_r6)
    ax.cla()
    ax.plot(rvals,energies,lw=3)
    ax.set_xlabel('R (nm)')
    ax.set_ylabel('Energy (kJ/mol)')
    ax.set_xlim(0, maxr)
    ax.set_ylim(-0.4,0.2)
    fig.canvas.draw()

# Generate the plots; evaluate the next box to generate
# sliders that allow sigma and epsilon to be varied
fig,ax = plt.subplots(1,1,figsize=(4,4))
showpotential(ax)

In [ ]:
# Evaluate this cell, then play with the sliders to see the effect of the well
# depth and radius on the shape of the potential curve for a dimer.
interact(showpotential,ax=fixed(ax),sigma=widgets.FloatSlider(min=0,max=0.5,step=.01,value=0.25),
         epsilon=widgets.FloatSlider(min=0,max=.3,step=.01,value=.05),);

Periodic Boundary Conditions
--------------------------------

Condensed phase simulations typically seek behavior of a given substance in the bulk.  However, the presence of container walls can cause significantly different behavior from the bulk, so it's desirable to avoid confining substances with hard walls.  One solution is to use periodic boundary conditions (PBC) whereby a unit cell is defined.  A particle leaving a face of the unit cell will then simply reappear in the opposing wall with the same velocity and momentum.  This is demonstrated below.

In [ ]:
rval = 0.5
dr = 0.02
boxlength = 1.5

fig, ax = plt.subplots(1,1,figsize=(4,4))
line, = ax.plot(rval, 0, 'ko')

def move_particle(i):
    global rval
    rval += dr
    # Impose PBC
    if(rval > boxlength):
        rval -= boxlength
    if(rval < 0):
        rval += boxlength
    line.set_xdata(rval)
    return line,

def init():
    line.set_xdata(rval)
    return line,

ani = animation.FuncAnimation(fig, move_particle, 500, init_func=init,interval=25, blit=True, repeat=False)
ax.set_xlim(0, boxlength)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])
plt.show()

The Minimum Image Convention
-----------------------------

Under PBC, we can think of the system as an infinite array of tessellated, identical unit cells; we are interested in studying the energy and properties of the primary unit cell in the presence of its cloned ("image") unit cells.  Given the infinite nature of the system, we're obviously not going to compute an infinite number of interactions, so we use a cutoff to truncate the calculation to a range of pairs whose distance is shorter than some threshold.

For each atom in the primary unit cell, we have to interact with all other atoms and images within the cutoff.  To make this easy to implement, we settle on the *minimum image convention*: each primary atom $i$ will interact with only the closest version of atom $j$, whether the closest version is a primary or an image.  This makes bookkeeping very simple, as we just have to shift interatomic distances to account for the PBC.  However, this imposes the restriction that the cutoff must be less than half of the box length, or we will include multiple copies of $j$ for a given $i$, which would require extra bookkeeping to correctly implement.

Below we setup a simple 2D box with particles on a regular grid, perturbed by a very small amount to break symmetry.  We then implement periodic boundary conditions, and a function to compute the LJ energy and its gradient.  The gradient expression is tested by numerical differentiation, which is always a good sanity check to perform.

In [ ]:
atoms_per_dim = 5
boxlength     = 2.0*atoms_per_dim    # atoms spaced by 2nm in each dimension
cutoff        = boxlength/2.0 - 1e-8 # Longest cutoff possible
epsilon       = 0.25               # kJ/mol
sigma         = 0.2                # nm

# intermediates just for convenience
cutoff2 = cutoff*cutoff
foureps = 4.0*epsilon
natoms = atoms_per_dim**2
pairs = np.array(list(itertools.combinations(range(natoms),2)))
iindices = pairs[:,0]
jindices = pairs[:,1]
sigmaij = 2.0*sigma # combine the sigma on each atom
sig2 = sigmaij*sigmaij
boxinv = 1.0/boxlength

# Initial coordinates; regular lattice
spacing = np.linspace(-boxlength/2, boxlength/2, atoms_per_dim, endpoint=False)
spacing += boxlength/(2*atoms_per_dim)
# Center the array
coords = np.array([np.repeat(spacing, atoms_per_dim), np.tile(spacing, atoms_per_dim)]).T
# Move the atoms very slightly away from their regular grid positions
np.random.seed(0)
coords += (np.random.rand(natoms,2)-0.5)*0.02


def apply_periodicity(dR):
    """ Apply minimum image convention. """
    dR -= boxlength*np.floor(dR*boxinv+0.5)
    return dR

def compute_energy_and_gradient_cutoff(coords):
    """ Use a simple cutoff method to compute the LJ energy and gradient. """
    forces = np.zeros((natoms, 2))
    # Apply minimum image convention
    dR = apply_periodicity(coords[jindices] - coords[iindices])
    r2 = np.einsum('ab,ab->a',dR, dR)
    r2inv = 1.0/r2
    # A cheesy way of applying the cutoffs
    r2inv[r2>cutoff2] = 0.0
    sig_r2 = sig2*r2inv
    sig_r6 = sig_r2*sig_r2*sig_r2
    sig_r12 = sig_r6*sig_r6
    repulsive  = foureps*sig_r12
    attractive = foureps*sig_r6
    pairenergies = repulsive - attractive
    pairforces = np.einsum('ab,a->ab',dR,6*r2inv*(repulsive+pairenergies))
    energy = pairenergies.sum()

    for n,(iind,jind) in enumerate(pairs):
        forces[iind] += pairforces[n]
        forces[jind] -= pairforces[n]
    return energy, forces
#
# Test the gradient by finite differences
#

# Displace the atoms in positive and negative directions for each
# degree of freedom and use the approximation
#
#         E(r+delta) - E(r-delta)
# grad =  -----------------------
#                2 delta
#
# to verify that the force expression has been coded up correctly.
fdgrad = np.zeros((natoms,2))
delta = 1e-6 # The step size can be varied
refE, refF = compute_energy_and_gradient_cutoff(coords)
for atom in range(natoms):
    # Plus x
    coords[atom,0] += delta
    Epx, Fpx = compute_energy_and_gradient_cutoff(coords)
    coords[atom,0] -= delta
    # Minus x
    coords[atom,0] -= delta
    Emx, Fmx = compute_energy_and_gradient_cutoff(coords)
    coords[atom,0] += delta
    # Plus y
    coords[atom,1] += delta
    Epy, Fpy = compute_energy_and_gradient_cutoff(coords)
    coords[atom,1] -= delta
    # Minus y
    coords[atom,1] -= delta
    Emy, Fmy = compute_energy_and_gradient_cutoff(coords)
    coords[atom,1] += delta
    
    fdgrad[atom] = [(Epx-Emx)/(2*delta), (Epy-Emy)/(2*delta)]

print("Analytic:")
print(refF)
print("Finite Difference:")
print(fdgrad)
print("Difference:")
print(refF-fdgrad)
print("Ratio:")
print(refF/fdgrad)

# Verify the energies and gradients are correct
assert np.allclose(-0.00370179210166, refE)
assert np.allclose(fdgrad, refF)

Running a trajectory
-------------------

The aim of molecular dynamics is to propagate a system in time, according to Newton's equations of motion, in increments of $\Delta$t.  To do this, we use the beautifully simple Verlet integrator.  Start by recognizing that the velocities are the time derivatives of the positions, $\mathbf{v}(t) = \mathbf{r}'(t)$, the accelerations are the second time derivatives, $\mathbf{a}(t) = \mathbf{r}''(t)$, and the jerk is the corresponding third derivative, $\mathbf{b}(t) = \mathbf{r}'''(t)$.  Now we can expand the positions as a power series in time, in forwards and backwards directions:

\begin{equation}
    \begin{split}
        \mathbf{r}(t+\Delta t) &= \mathbf{r}(t) + \mathbf{v}(t)\Delta t + \frac{1}{2}\mathbf{a}(t)\Delta t^2  + \frac{1}{6}\mathbf{b}(t)\Delta t^3 + \ldots \\
        \\
        \mathbf{r}(t-\Delta t) &= \mathbf{r}(t) - \mathbf{v}(t)\Delta t + \frac{1}{2}\mathbf{a}(t)\Delta t^2  - \frac{1}{6}\mathbf{b}(t)\Delta t^3 + \ldots \\
    \end{split}
\end{equation}

Adding these expressions and rearranging gives us the Verlet integration algorithm:

\begin{equation}
    \mathbf{r}(t+\Delta t) = 2\mathbf{r}(t) - \mathbf{r}(t-\Delta t) + \mathbf{a}(t)\Delta t^2  + \mathcal{O}(\Delta t^4).
\end{equation}

There are a few noteworthy features of this expression.  First, we note that the velocities are not needed (they can be computed by finite differences of the current, previous and next positions) although variants of the algorithm do exist that use velocities.  We need the previous step's position information, which means that we just use the simple forwards power series expansion above to get started.  Note that the next and previous positions appear in a symmetrical way, which means that this algorithm can be run in reverse to recover the starting positions.

We want the potential energy function, $U(\mathbf{r})$, to drive the dynamics.  To make that connection, we remember the result $F_i(\mathbf{r})=m_i\mathbf{a}_i(t)$ from classical mechanics, and rearrange 

\begin{equation}
    \mathbf{a}_i(t) = \frac{F_i(\mathbf{r})}{m_i} = -\frac{1}{m_i}\frac{\partial U(\mathbf{r})}{\partial \mathbf{r}_i},
\end{equation}

remembering that the $i$ subscript labels each atom.  Integrating the equations this way should maintain total energy, because no energy or particles are exchanged with the surroundings; in statistical mechanics this is known as a microcanonical ensemble, or NVE to reflect the that that number of particles, volume and energy are all constant.  In the microcanonical ensemble, although total energy is conserved, kinetic and potential may interconvert through time.

To test different simulation conditions easily, we encapsulate all of the simulation machinery we've developed so far into a class, which can take simulation setup as input to allow us to experiment with different conditions.

In [ ]:
Rgas = 0.0083144621 # Molar gas constant in kJ/(mol K)

class MDSimulation(object):
    """ A class to encapsulate coordinate and potential energy information about
        a Lennard-Jones simulation in a cubic unit cell.  Plotting functions are
        provided to keep track of the simulation in real time, using Matplotlib. """
    def __init__(self, sigma, epsilon, mass, temp, dt, atoms_per_dim, boxlength, cutoff, nsteps):
        """ Set up the simulation conditions and LJ potential information. """
        self.sigma         = sigma
        self.epsilon       = epsilon
        self.mass          = mass
        self.dt            = dt
        self.T             = temp
        self.RT            = Rgas*self.T
        self.atoms_per_dim = atoms_per_dim
        self.boxlength     = boxlength
        self.cutoff        = cutoff
        self.nsteps        = nsteps
    
        # intermediates just for convenience
        self.cutoff2   = self.cutoff**2
        self.foureps   = 4.0*self.epsilon
        self.natoms    = self.atoms_per_dim**2
        self.pairs     = np.array(list(itertools.combinations(range(self.natoms),2)))
        self.iindices  = self.pairs[:,0]
        self.jindices  = self.pairs[:,1]
        self.sigmaij   = 2.0*self.sigma # combine the sigma on each atom
        self.sig2      = self.sigmaij**2
        self.boxinv    = 1.0/self.boxlength
        self.minv      = 1.0/self.mass
        
        if(cutoff >= boxlength/2.0):
            raise ValidationError("The cutoff must be less than boxlength/2")

        np.random.seed(0)
        self.init_coords()
        self.last_coords = np.copy(self.coords)
        # Generate velocities
        velocities = np.sqrt(self.RT/self.mass)*np.random.normal(size=(self.natoms,2))
        KE = 0.5*self.mass*np.sum(np.square(velocities))
        # Take a step x1 = x0 + v dt + 1/2 a dt^2
        PE, gradient = self.compute_energy_and_gradient(self.coords)
        
        self.timestep = 0.0
        self.timeseries = [ self.timestep ]
        self.PEs = [ PE ]
        self.KEs = [ KE ]
        self.TEs = [ KE + PE ]
        a = -self.minv*gradient
        self.coords += velocities*self.dt + 0.5*self.dt*self.dt*a
        
        # Plot objects
        self.fig, (self.enerplot, self.coordplot) = plt.subplots(1,2,figsize=(8.2,4))
        self.coorddata, = self.coordplot.plot(self.coords[:,0], self.coords[:,1], 'ko')
        self.kline, = self.enerplot.plot(0, 0, 'bo', ms=0.5, label='kinetic')
        self.vline, = self.enerplot.plot(0, 0, 'ro', ms=0.5, label='potential')
        self.eline, = self.enerplot.plot(0, 0, 'ko', ms=0.5, label='total')
        self.coordplot.set_xlim(-self.boxlength/2.0, self.boxlength/2.0)
        self.coordplot.set_ylim(-self.boxlength/2.0, self.boxlength/2.0)
        self.coordplot.set_aspect('equal')
        self.coordplot.get_xaxis().set_ticks([])
        self.coordplot.get_yaxis().set_ticks([])
        self.coordplot.set_ylabel('simulation box')
        self.enerplot.set_xlim(0, nsteps*self.dt)
        self.enerplot.set_xlabel("Time (ps)")
        self.enerplot.set_ylabel("Energy (kJ/mol)")
        self.enerplot.legend()
        self.animator = None


    def compute_energy_and_gradient(self, coords):
        """ Method to compute the LJ energy and gradient, which can be overloaded. """
        return self.compute_energy_and_gradient_cutoff(coords)
    
    def propagate(self):
        """ Update the particle positions, using Verlet integration. """
        # Take a step r(t+dt) = 2 r(t) - r(t-dt) + dt^2 a(t)
        PE, gradient = self.compute_energy_and_gradient(self.coords)
        a = -self.minv*gradient
        newcoords = 2*self.coords - self.last_coords + self.dt*self.dt*a
        v = (newcoords - self.last_coords)/(2*self.dt)
        KE = 0.5*self.mass*np.sum(np.square(v))
        # Update coordinates
        self.last_coords = self.coords.copy()
        self.coords = newcoords.copy()
        displaycoords = self.apply_periodicity(self.coords.copy())
        self.coorddata.set_data(displaycoords[:,0], displaycoords[:,1])  # update the data
        self.timestep += self.dt
        self.timeseries.append(self.timestep)
        self.PEs.append(PE)
        self.KEs.append(KE)
        self.TEs.append(PE + KE)
        self.vline.set_data(self.timeseries, self.PEs)
        self.kline.set_data(self.timeseries, self.KEs)
        self.eline.set_data(self.timeseries, self.TEs)
        en = (PE, KE, PE+KE)
        minlim = np.min((1.2*np.min(en), self.enerplot.get_ylim()[0]))
        maxlim = np.max((1.2*np.max(en), self.enerplot.get_ylim()[1]))
        self.enerplot.set_ylim(minlim, maxlim)
        return self.coorddata,
    
    def init_coords(self):
        """ Set up the initial simulation coordinates on a regular grid, perturbed slightly. """
        # Initial coordinates; regular lattice
        spacing = np.linspace(-self.boxlength/2.0, self.boxlength/2.0, self.atoms_per_dim, endpoint=False)
        spacing += self.boxlength/(2.0*self.atoms_per_dim)
        # Center the array
        self.coords = np.array([np.repeat(spacing, self.atoms_per_dim), np.tile(spacing, self.atoms_per_dim)]).T
        # Move the atoms very slightly away from their regular grid positions
        self.coords += 0.2*(np.random.rand(self.natoms,2)-0.5)

    def testgrad(self, show=0):
        """ Use finite differences to verify that the energy and gradient expressions are consistent. """
        fdgrad = np.zeros((self.natoms,2))
        delta = 1e-6 # The step size can be varied
        refE, refF = self.compute_energy_and_gradient(self.coords)
        for atom in range(self.natoms):
            # Plus x
            self.coords[atom,0] += delta
            Epx, Fpx = self.compute_energy_and_gradient(self.coords)
            self.coords[atom,0] -= delta
            # Minus x
            self.coords[atom,0] -= delta
            Emx, Fmx = self.compute_energy_and_gradient(self.coords)
            self.coords[atom,0] += delta
            # Plus y
            self.coords[atom,1] += delta
            Epy, Fpy = self.compute_energy_and_gradient(self.coords)
            self.coords[atom,1] -= delta
            # Minus y
            self.coords[atom,1] -= delta
            Emy, Fmy = self.compute_energy_and_gradient(self.coords)
            self.coords[atom,1] += delta
            #
            fdgrad[atom] = [(Epx-Emx)/(2*delta), (Epy-Emy)/(2*delta)]
        if show:
            print("Analytic:")
            print(refF)
            print("Finite Difference:")
            print(fdgrad)
            print("Difference:")
            print(refF-fdgrad)        
            print("Ratio:")
            print(refF/fdgrad) 
        # Confirm the gradients are correct
        assert np.allclose(refF, fdgrad)

    def apply_periodicity(self, dR):
        """ Apply minimum image convention to an internuclear vector. """
        dR -= self.boxlength*np.floor(dR*self.boxinv+0.5)
        return dR

    def compute_energy_and_gradient_cutoff(self, coords):
        """ Compute the LJ energy and gradient, using a simple truncation scheme. """
        forces = np.zeros((self.natoms, 2))
        # Apply minimum image convention
        dR = self.apply_periodicity(coords[self.jindices] - coords[self.iindices])
        r2 = np.einsum('ab,ab->a',dR, dR)
        r2inv = 1.0/r2
        # A cheesy way of applying the cutoffs
        r2inv[r2>self.cutoff2] = 0.0
        sig_r2 = self.sig2*r2inv
        sig_r6 = sig_r2*sig_r2*sig_r2
        sig_r12 = sig_r6*sig_r6
        repulsive  = self.foureps*sig_r12
        attractive = self.foureps*sig_r6
        pairenergies = repulsive - attractive
        pairforces = np.einsum('ab,a->ab',dR,6*r2inv*(repulsive+pairenergies))
        energy = pairenergies.sum()
        for n,(iind,jind) in enumerate(self.pairs):
            forces[iind] += pairforces[n]
            forces[jind] -= pairforces[n]
        return energy, forces
    
    def run_simulation(self):
        """ A hook to propagate the trajectory and animate in real time using
            Matplotlib's Funcanimation routine. """
        def take_step(i, self):
            # A dummy function because FuncAnimation wants to pass an integer in
            self.propagate()
        # N.B. The animator must be persistent, which is why we cache it as a class variable here
        self.animator = animation.FuncAnimation(self.fig, take_step, self.nsteps, fargs=(self,),
                                                interval=25, repeat=False)

## Run a short simulation with a short cutoff

This contrived example shows what can happen if insufficient cutoffs are used in a simulation.  First, we run with a very short 0.5 nm cutoff.

In [ ]:
sim = MDSimulation(sigma=0.27, epsilon=0.3, mass=2, temp=300, dt=0.005, atoms_per_dim=5, boxlength=3,
                   cutoff=0.5, nsteps=400)
# Verify that the first energy is correct
assert np.allclose(13.7148290021, sim.compute_energy_and_gradient(sim.coords)[0])
# Verify that the first gradient is correct
sim.testgrad(show=0)
sim.run_simulation()

## Run a short simulation with a long cutoff

Running with a longer cutoff restores energy conservation.  Although kinetic and potential energy are interchanged, the total energy stays roughly constant with a 1.2 nm cutoff.  However, the computational cost is increased, as more pairs must be considered at each time step.

In [ ]:
sim = MDSimulation(sigma=0.27, epsilon=0.3, mass=2, temp=300, dt=0.005, atoms_per_dim=5, boxlength=3, cutoff=1.2, nsteps=500)
# Verify that the first energy is correct
assert np.allclose(2.52645225873, sim.compute_energy_and_gradient(sim.coords)[0])
# Verify that the first gradient is correct
sim.testgrad(show=0)
sim.run_simulation()

Dealing with cutoff errors
--------------------------
The origin of the energy deviations is a step in the potential, at the cutoff.  As atoms cross this discontinuity, energy is gained or lost, so the microcanonical ensemble is not properly preserved.  The longer cutoffs remedied the situation, because the magnitude of the step is diminished.  To visualize this, we plot the truncated LJ potential below.

In [ ]:
def showpotential(ax, sigma=0.25, epsilon=0.11, cutoff=0.8):
    npts = 200
    sigmapair = 2*sigma # combine the sigmas on each atom
    # Go up to 1.5nm, using npts points
    maxr = 1.5
    rvals = np.linspace(0.3, maxr, npts)
    sig_r = sigmapair / rvals
    sig_r[rvals>cutoff] = 0
    sig_r2 = np.square(sig_r)
    sig_r6 = sig_r2*sig_r2*sig_r2
    sig_r12 = sig_r6*sig_r6
    energies = 4*epsilon*(sig_r12 - sig_r6)
    ax.cla()
    ax.plot(rvals,energies,lw=3)
    ax.set_xlabel('R (nm)')
    ax.set_ylabel('Energy (kJ/mol)')
    ax.set_xlim(0, maxr)
    ax.set_ylim(-0.4,0.2)
    fig.canvas.draw()

# Plot the initial LJ potential.  Running the next box will
# provide slider to vary the different parameters.
fig,ax = plt.subplots(1,1,figsize=(3,3))
showpotential(ax)

In [ ]:
# Evaluate this cell, then play with the sliders to see the effect of the well
# depth, radius and cutoff on the shape of the potential curve for a dimer.
interact(showpotential,ax=fixed(ax),sigma=widgets.FloatSlider(min=0,max=0.5,step=.01,value=0.25),
         epsilon=widgets.FloatSlider(min=0,max=.3,step=.01,value=.11),
         cutoff=widgets.FloatSlider(min=0,max=1.5,step=.05,value=0.8));

Switching techniques
---------------------

To avoid the 'brute force' remedy of using longer cutoffs, we can instead introduce a switching function to smooth the potential around the cutoff.  To do this, we define a window immediately inside the cutoff and smoothly attenuate the potential from its value at the start of the window, to zero at the end of the window, which occurs at the cutoff.  We now plot the switched LJ potential.

In [ ]:
def showpotential(ax, sigma=0.25, epsilon=0.11, cutoff=0.8, window=0.2):
    """ Plot a switched LJ potential, provided the parameters and the switching window. """
    npts = 200
    sigmapair = 2*sigma # combine the sigmas on each atom
    # Go up to 15A, using npts points
    maxr = 1.5
    rvals = np.linspace(0.3, maxr, npts)
    ron2 = (cutoff-window)**2
    roff2 = cutoff**2
    r2vals = np.square(rvals)
    switch = np.where(r2vals>ron2, (roff2 + 2*r2vals - 3*ron2)*(roff2-r2vals)**2/(roff2-ron2)**3, 1)
    # Introduce cutoff
    switch[rvals>cutoff] = 0
    # Introduce switch
    sig_r = (sigmapair/rvals)
    sig_r2 = np.square(sig_r)
    sig_r6 = sig_r2*sig_r2*sig_r2
    sig_r12 = sig_r6*sig_r6
    energies = 4*epsilon*switch*(sig_r12 - sig_r6)
    ax.cla()
    ax.plot(rvals,energies,lw=3)
    ax.set_xlabel('R (nm)')
    ax.set_ylabel('Energy (kJ/mol)')
    ax.set_xlim(0, maxr)
    ax.set_ylim(-0.4,0.2)
    fig.canvas.draw()

# Plot the switched LJ potential; run the next box to get
# sliders to vary the parameters.
fig,ax = plt.subplots(1,1,figsize=(3,3))
showpotential(ax)

In [ ]:
# Evaluate this cell, then play with the sliders to see the effect of the well
# depth, radius, window and cutoff on the shape of the potential curve for a dimer.
interact(showpotential,ax=fixed(ax),sigma=widgets.FloatSlider(min=0,max=0.5,step=.01,value=0.25),
         epsilon=widgets.FloatSlider(min=0,max=.3,step=.01,value=.11),
         cutoff=widgets.FloatSlider(min=0,max=1.5,step=.05,value=0.8),
         window=widgets.FloatSlider(min=0.001,max=0.4,step=.05,value=0.2));

Updating the simulation class
------------------------------

We can modify the simulation class by deriving a new type from it and overriding the `compute_energy_and_gradient` method, replacing it with the switched LJ function;  all other functions will be the same as those in the parent `MDSimulation` class.

In [ ]:
class MDSimulationSwitch(MDSimulation):
    """ Overloaded version of MDSimulation to use a switched LJ potential. """
    
    def __init__(self, sigma, epsilon, mass, temp, dt, atoms_per_dim, boxlength, cutoff, window, nsteps):
        """ Instantiate the underlying MDSimulation object, with additional switching window information. """
        self.window = window
        super(MDSimulationSwitch, self).__init__(sigma, epsilon, mass, temp, dt, atoms_per_dim, boxlength, cutoff, nsteps)

        
    def compute_energy_and_gradient(self, coords):
        """ Compute the LJ energy and gradient, with optional switching to smooth the potential. """
        if self.window == 0:
            return self.compute_energy_and_gradient_cutoff(coords)
        else:
            return self.compute_energy_and_gradient_switch(coords)
            
    def compute_energy_and_gradient_switch(self, coords):
        """ Compute the switched LJ potential energy and gradient. """
        ron2 = (self.cutoff-self.window)**2
        roff2 = self.cutoff2
        denom = 1.0 / (roff2 - ron2)**3
        forces = np.zeros((self.natoms, 2))
        # Apply minimum image convention
        dR = self.apply_periodicity(coords[self.jindices] - coords[self.iindices])
        r2 = np.einsum('ab,ab->a',dR, dR)
        r2inv = 1.0/r2
        # Switching functions
        switch  = np.where(r2>ron2, (roff2 + 2.0*r2 - 3.0*ron2)*(roff2 - r2)**2 * denom, 1.0)
        dswitch = np.where(r2>ron2,  -12.0*(r2 - roff2)*(r2 - ron2)  * denom, 0.0)
        # Introduce cutoff
        switch[r2>self.cutoff2]  = 0.0
        dswitch[r2>self.cutoff2] = 0.0
        sig_r2 = self.sig2*r2inv
        sig_r6 = sig_r2*sig_r2*sig_r2
        sig_r12 = sig_r6*sig_r6
        rep  = self.foureps*sig_r12
        att  = self.foureps*sig_r6
        pairenergies = switch*(rep - att)
        pairforces = np.einsum('ab,a->ab', dR, 6.0*r2inv*switch*(2.0*rep-att) + dswitch*(rep-att))
        energy = pairenergies.sum()
        for n,(iind,jind) in enumerate(self.pairs):
            forces[iind] += pairforces[n]
            forces[jind] -= pairforces[n]
        return energy, forces

## Run the same short cutoff simulation, but with a switching function
Hint: to run the switch-free simulation again, set the window to zero.

In [ ]:
sim = MDSimulationSwitch(sigma=0.27, epsilon=0.3, mass=2, temp=300, dt=0.005, atoms_per_dim=5,
                         boxlength=3, cutoff=0.5, nsteps=500, window=0.1)
# Verify that the first energy is correct
assert np.allclose(5.76688248814, sim.compute_energy_and_gradient(sim.coords)[0])
# Verify that the first gradient is correct
sim.testgrad(show=0)
sim.run_simulation()

By introducing switching, energy is conserved even with a 0.5 nm cutoff.

References
=====

1. M. P. Allen and D. J. Tildesley, *Computer Simulation of Liquids*, Oxford University Press, New York 1987.
2. D. Frenkel and B. Smit, *Understanding Molecular Simulation: From Algorithms to Applications*, Academic Press, San Diego 1996.
3. P. J. Steinbach and B. R. Brooks, *J. Comp. Chem.*, 15 667 (1994).